In [10]:
# Comprueba si tienes instalado Pytorch en tu Sistema WSl2 con ROcm:
import torch
x = torch.rand(5,3)
print(x)
#torch.cuda.is_available()
%matplotlib inline

tensor([[0.1447, 0.1108, 0.4910],
        [0.8418, 0.4819, 0.9591],
        [0.5382, 0.3743, 0.5309],
        [0.7117, 0.7145, 0.7193],
        [0.7661, 0.6256, 0.6781]])


## Trabajando con datos
PyTorch manipula dos tipos primitivos de datos:
``torch.utils.data.DataLoader`` and ``torch.utils.data.Dataset``.
``Dataset`` almacena sus muestras con sus etiquetas correspondientes, y ``DataLoader`` crea una iteracion en el ``Dataset``.


In [7]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

PyTorch ofrece bibliotecas específicas de dominio como ``TorchText``, ``TorchVision`` y ``TorchAudio``, todas las cuales incluyen conjuntos de datos. Aqui utilizamos un conjunto de datos ``TorchVision``.

El módulo torchvision.datasets contiene objetos Dataset para muchos datos de visión del mundo real como CIFAR, COCO. En este tutorial, utilizamos el conjunto de datos FashionMNIST. Cada TorchVision Dataset incluye dos argumentos: transform y target_transform para modificar las muestras y las etiquetas respectivamente.



In [8]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100.0%


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100.0%


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100.0%


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100.0%

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



Pasamos el conjunto de datos como argumento a DataLoader. Esto envuelve un iterable sobre nuestro conjunto de datos, y soporta la carga automática de datos por lotes, muestreo, barajado y multiproceso. Aquí definimos un tamaño de lote de 64, es decir, cada elemento en el iterable dataloader devolverá un lote de 64 características y etiquetas.



In [11]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


--------------




## Creando modelos
Para definir una red neuronal en PyTorch, creamos una clase que hereda de [nn.Module]. Definimos las capas de la red en la función __init__ y especificamos cómo pasarán los datos a través de la red en la función forward. Para acelerar las operaciones en la red neuronal, la movemos a la GPU o MPS si está disponible.


In [12]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## Optimizando los parametros del modelo
Para entrenar un modelo, necesitamos la funcion: [loss function]
y tambien: [optimizer]



In [13]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

En un único bucle de entrenamiento, el modelo realiza predicciones sobre el conjunto de datos de entrenamiento (alimentado por lotes), y retropropaga el error de predicción para ajustar los parámetros del modelo.



In [14]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

También comprobamos el rendimiento del modelo con el conjunto de datos de prueba para asegurarnos de que está aprendiendo.



In [15]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

El proceso de entrenamiento se lleva a cabo a lo largo de varias iteraciones (*epochs*). Durante cada epoch, el modelo aprende
parámetros para hacer mejores predicciones. Imprimimos la precisión y la pérdida del modelo en cada época.
y la pérdida disminuye con cada epoch.



In [16]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("¡Trabajo completado!")

Epoch 1
-------------------------------
loss: 2.287865  [   64/60000]
loss: 2.280092  [ 6464/60000]
loss: 2.256556  [12864/60000]
loss: 2.262291  [19264/60000]
loss: 2.234435  [25664/60000]
loss: 2.199603  [32064/60000]
loss: 2.211268  [38464/60000]
loss: 2.168447  [44864/60000]
loss: 2.174129  [51264/60000]
loss: 2.142170  [57664/60000]
Test Error: 
 Accuracy: 39.1%, Avg loss: 2.134272 

Epoch 2
-------------------------------
loss: 2.144091  [   64/60000]
loss: 2.134423  [ 6464/60000]
loss: 2.075452  [12864/60000]
loss: 2.102607  [19264/60000]
loss: 2.026638  [25664/60000]
loss: 1.972186  [32064/60000]
loss: 2.004623  [38464/60000]
loss: 1.918372  [44864/60000]
loss: 1.931993  [51264/60000]
loss: 1.861191  [57664/60000]
Test Error: 
 Accuracy: 51.8%, Avg loss: 1.853627 

Epoch 3
-------------------------------
loss: 1.891190  [   64/60000]
loss: 1.858196  [ 6464/60000]
loss: 1.741704  [12864/60000]
loss: 1.789665  [19264/60000]
loss: 1.655540  [25664/60000]
loss: 1.626629  [32064/600

--------------




## Guardando modelos
Una forma común de guardar un modelo es serializar el diccionario de estado interno (que contiene los parámetros del modelo).



In [17]:
torch.save(model.state_dict(), "model.pth")
print("Guardando el estado del modelo al archivo-> model.pth")

Guardando el estado del modelo al archivo-> model.pth


## Cargando modelos

El proceso para cargar un modelo incluye recrear la estructura del modelo y cargar
el diccionario de estados.



In [18]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

Este modelo puede utilizarse ahora para hacer predicciones.



In [19]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
